The observed trends are:
#1 Students in high spending schools performed low
#2 Students in low budget spending schools performed high
#3 Chartered School Performed high and district school performed low
#4 Large size schools performed low

In [2]:
#dependencies
import numpy as np
import pandas as pd


In [3]:
#Reading csv files in pandas
schools_df= pd.read_csv("C:\\Users\\abu\\PythonData\\PyCitySchools\\raw_data\\schools_complete.csv")
students_df=pd.read_csv("C:\\Users\\abu\\PythonData\\PyCitySchools\\raw_data\\students_complete.csv")

In [6]:
#Checking the head and shape of the data frames
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [8]:
#checking the size of the student data frame
students_df.shape

(39170, 7)

### District Summary

In [5]:
#identifying students passing math and reading with passing score of 70
students_df["%Passing Reading"]=students_df["reading_score"]>=70
students_df["%Passing Math"]=students_df["math_score"]>=70

In [6]:
#calculating percent passing reading
PassingReading=students_df["%Passing Reading"].sum()/len(students_df["name"])*100

In [7]:
#Calculating percent passing math
PassingMath=students_df["%Passing Math"].sum()/len(students_df["name"])*100

In [8]:
#calculating overall passing rate
OverallPassing= (students_df["%Passing Math"].sum() + students_df["%Passing Reading"].sum())/2/len(students_df["name"])*100

In [13]:
#Storing  all to a dictionary
District_dic= {
                                "Total School":[schools_df["name"].count()],
                                "Total Students":[students_df["name"].count()],
                                "Total Budget":[schools_df["budget"].sum()],
                                "Average Math Score ":[students_df["math_score"].mean()],
                                "Average Reading Score":[students_df["reading_score"].mean()],
                                "%Passing Math":[PassingMath],
                                "%Passing Reading":[PassingReading],
                                "Overall Passing Rate":[OverallPassing]}

In [14]:
#converting the dictionary to a data frame
District_Summary=pd.DataFrame(District_dic,columns=pd.Index(["Total School","Total Students","Total Budget","Average Math Score",
                                                             "Average Reading Score","%Passing Math","%Passing Reading","Overall Passing Rate"]))

In [57]:
#Snaphsot of the District Summary
District_Summary

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Passing Rate
0,15,39170,24649428,NaN,81.87784,74.980853,85.805463,80.393158


In [9]:
#renaming the name column of schools dataframe to school 
schools_df=schools_df.rename(columns={"name":"school"})

In [10]:
#combining the school and the student data frame using merge
District_School= pd.merge(schools_df,students_df,how='inner',on='school')

In [60]:
District_School.head()

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score,%Passing Reading,%Passing Math
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,False,True
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,True,False
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,True,False
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,False,False
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,True,True


In [36]:
#Grouping the district school data with school name
District_group=District_School.groupby("school").agg({"type":"first",
                                                                     "name":"count",
                                                                     "budget":"first",
                                                                     "math_score":"mean",
                                                                     "reading_score":["mean"],
                                                                     "%Passing Reading":["sum"],
                                                                     "%Passing Math":sum })

In [37]:
District_group.head()

,type,name,budget,math_score,reading_score,%Passing Reading,%Passing Math
,first,count,first,mean,mean,sum,sum
school,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,4077.0,3318.0
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1803.0,1749.0
Figueroa High School,District,2949,1884411,76.711767,81.158020,2381.0,1946.0
Ford High School,District,2739,1763916,77.102592,80.746258,2172.0,1871.0
Griffin High School,Charter,1468,917500,83.351499,83.816757,1426.0,1371.0


In [38]:
#removing the groupby aggregate column
District_group.columns=District_group.columns.droplevel(1)

In [47]:
#Adding the overall passing column
District_group["Overall Passing Rate"]=(District_group["%Passing Reading"]+District_group["%Passing Math"])/2

,school,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,%Passing Reading,%Passing Math,Overall Passing Rate
0,Bailey High School,[District],4976,[3124928],77.048432,81.033963,4077.0,3318.0,3697.5
1,Cabrera High School,[Charter],1858,[1081356],83.061895,83.975780,1803.0,1749.0,1776.0
2,Figueroa High School,[District],2949,[1884411],76.711767,81.158020,2381.0,1946.0,2163.5
3,Ford High School,[District],2739,[1763916],77.102592,80.746258,2172.0,1871.0,2021.5
4,Griffin High School,[Charter],1468,[917500],83.351499,83.816757,1426.0,1371.0,1398.5
5,Hernandez High School,[District],4635,[3022020],77.289752,80.934412,3748.0,3094.0,3421.0
6,Holden High School,[Charter],427,[248087],83.803279,83.814988,411.0,395.0,403.0
7,Huang High School,[District],2917,[1910635],76.629414,81.182722,2372.0,1916.0,2144.0
8,Johnson High School,[District],4761,[3094650],77.072464,80.966394,3867.0,3145.0,3506.0
9,Pena High School,[Charter],962,[585858],83.839917,84.044699,923.0,910.0,916.5


In [40]:
#Calculating the percentage of passing math
District_group["%Passing Math"]=District_group["%Passing Math"]/District_group["name"]*100

In [41]:
#Calculating the percentage of passing reading
District_group["%Passing Reading"]=District_group["%Passing Reading"]/District_group["name"]*100

In [42]:
#Calculating the percetage of overall passing rate
District_group["Overall Passing Rate"]=District_group["Overall Passing Rate"]

In [43]:
#Calculating the buget per student
District_group["Per Student Budget"]=District_group["budget"]/District_group["name"]*100

### School Summary

In [48]:
#Renaming the columns of the school Summary
District_group.rename(columns={"type":"School Type","name":"Total Students","budget":"Total School Budget","math_score":"Average Math Score","reading_score":"Average Reading Score","%Passing Reading":"%Passing Reading ","%Passing Math":"%Passing Math","Overall Passing Rate":"Overall Passing Rate"})

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,%Passing Reading,%Passing Math,Overall Passing Rate,Per Student Budget
school,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,81.933280,66.680064,74.306672,62800.0
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,97.039828,94.133477,95.586652,58200.0
Figueroa High School,District,2949,1884411,76.711767,81.158020,80.739234,65.988471,73.363852,63900.0
Ford High School,District,2739,1763916,77.102592,80.746258,79.299014,68.309602,73.804308,64400.0
Griffin High School,Charter,1468,917500,83.351499,83.816757,97.138965,93.392371,95.265668,62500.0
Hernandez High School,District,4635,3022020,77.289752,80.934412,80.862999,66.752967,73.807983,65200.0
Holden High School,Charter,427,248087,83.803279,83.814988,96.252927,92.505855,94.379391,58100.0
Huang High School,District,2917,1910635,76.629414,81.182722,81.316421,65.683922,73.500171,65500.0
Johnson High School,District,4761,3094650,77.072464,80.966394,81.222432,66.057551,73.639992,65000.0


,type,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate,Per Student Budget
school,,,,,,,,,
Bailey High School,[District],4976,[3124928],77.048432,81.033963,81.933280,66.680064,74.306672,[62800.0]
Cabrera High School,[Charter],1858,[1081356],83.061895,83.975780,97.039828,94.133477,95.586652,[58200.0]
Figueroa High School,[District],2949,[1884411],76.711767,81.158020,80.739234,65.988471,73.363852,[63900.0]
Ford High School,[District],2739,[1763916],77.102592,80.746258,79.299014,68.309602,73.804308,[64400.0]
Griffin High School,[Charter],1468,[917500],83.351499,83.816757,97.138965,93.392371,95.265668,[62500.0]


### Top Performing School

In [49]:
#Idetifying the top performing schools by overall Passing rate
Top_Performing_School=District_group.sort_values(["Overall Passing Rate"], ascending=False)

In [50]:
Top_Performing_School.head()

,type,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate,Per Student Budget
school,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,97.039828,94.133477,95.586652,58200.0
Thomas High School,Charter,1635,1043130,83.418349,83.848930,97.308869,93.272171,95.290520,63800.0
Pena High School,Charter,962,585858,83.839917,84.044699,95.945946,94.594595,95.270270,60900.0
Griffin High School,Charter,1468,917500,83.351499,83.816757,97.138965,93.392371,95.265668,62500.0
Wilson High School,Charter,2283,1319574,83.274201,83.989488,96.539641,93.867718,95.203679,57800.0


### low performing school

In [51]:
#identifying the low performing schools by overall passing rate
low_performing_school=District_group.sort_values(["Overall Passing Rate"], ascending=True)

In [52]:
low_performing_school.head()

,type,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate,Per Student Budget
school,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.842711,80.744686,80.220055,66.366592,73.293323,63700.0
Figueroa High School,District,2949,1884411,76.711767,81.158020,80.739234,65.988471,73.363852,63900.0
Huang High School,District,2917,1910635,76.629414,81.182722,81.316421,65.683922,73.500171,65500.0
Johnson High School,District,4761,3094650,77.072464,80.966394,81.222432,66.057551,73.639992,65000.0
Ford High School,District,2739,1763916,77.102592,80.746258,79.299014,68.309602,73.804308,64400.0


### Math Score by grade

In [53]:
#grouping the district school by school and grade
Math_Score_Per_Grade=District_School.groupby(['school',"grade"])["math_score"].mean()

In [54]:
Math_Score_Per_Grade

school                 grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

In [55]:
type(Math_Score_Per_Grade)

pandas.core.series.Series

In [56]:
#converting the groupby to data frame
Math_Score_Per_Grade_df=pd.DataFrame(Math_Score_Per_Grade)

In [57]:
Math_Score_Per_Grade_df

math_score
school                grade            
Bailey High School    10th    76.996772
                      11th    77.515588
                      12th    76.492218
                      9th     77.083676
Cabrera High School   10th    83.154506
                      11th    82.765560
                      12th    83.277487
                      9th     83.094697
Figueroa High School  10th    76.539974
                      11th    76.884344
                      12th    77.151369
                      9th     76.403037
Ford High School      10th    77.672316
                      11th    76.918058
                      12th    76.179963
                      9th     77.361345
Griffin High School   10th    84.229064
                      11th    83.842105
                      12th    83.356164
                      9th     82.044010
Hernandez High School 10th    77.337408
                      11th    77.136029
                      12th    77.186567
                      9th     77.438495
Holden High School    10th    83.429825
                      11th    85.000000
                      12th    82.855422
                      9th     83.787402
Huang High School     10th    75.908735
                      11th    76.446602
                      12th    77.225641
                      9th     77.027251
Johnson High School   10th    76.691117
                      11th    77.491653
                      12th    76.863248
                      9th     77.187857
Pena High School      10th    83.372000
                      11th    84.328125
                      12th    84.121547
                      9th     83.625455
Rodriguez High School 10th    76.612500
                      11th    76.395626
                      12th    77.690748
                      9th     76.859966
Shelton High School   10th    82.917411
                      11th    83.383495
                      12th    83.778976
                      9th     83.420755
Thomas High School    10th    83.087886
                      11th    83.498795
                      12th    83.497041
                      9th     83.590022
Wilson High School    10th    83.724422
                      11th    83.195326
                      12th    83.035794
                      9th     83.085578
Wright High School    10th    84.010288
                      11th    83.836782
                      12th    83.644986
                      9th     83.264706

In [29]:
Math_Score_Per_Grade_df.set_index("school")

KeyError: 'school'

In [58]:
Math_Score_Per_Grade_df.index()

TypeError: 'MultiIndex' object is not callable

### Reading Score Per Grade

In [59]:
#grouping the district school by school and grade
Reading_Score_Per_Grade=District_School.groupby(["school","grade"])["reading_score"].mean()

In [60]:
#changing the groupby series to data frame
Reading_Score_Per_Grade_df=pd.DataFrame(Reading_Score_Per_Grade)

In [61]:
Reading_Score_Per_Grade_df

reading_score
school                grade               
Bailey High School    10th       80.907183
                      11th       80.945643
                      12th       80.912451
                      9th        81.303155
Cabrera High School   10th       84.253219
                      11th       83.788382
                      12th       84.287958
                      9th        83.676136
Figueroa High School  10th       81.408912
                      11th       80.640339
                      12th       81.384863
                      9th        81.198598
Ford High School      10th       81.262712
                      11th       80.403642
                      12th       80.662338
                      9th        80.632653
Griffin High School   10th       83.706897
                      11th       84.288089
                      12th       84.013699
                      9th        83.369193
Hernandez High School 10th       80.660147
                      11th       81.396140
                      12th       80.857143
                      9th        80.866860
Holden High School    10th       83.324561
                      11th       83.815534
                      12th       84.698795
                      9th        83.677165
Huang High School     10th       81.512386
                      11th       81.417476
                      12th       80.305983
                      9th        81.290284
Johnson High School   10th       80.773431
                      11th       80.616027
                      12th       81.227564
                      9th        81.260714
Pena High School      10th       83.612000
                      11th       84.335938
                      12th       84.591160
                      9th        83.807273
Rodriguez High School 10th       80.629808
                      11th       80.864811
                      12th       80.376426
                      9th        80.993127
Shelton High School   10th       83.441964
                      11th       84.373786
                      12th       82.781671
                      9th        84.122642
Thomas High School    10th       84.254157
                      11th       83.585542
                      12th       83.831361
                      9th        83.728850
Wilson High School    10th       84.021452
                      11th       83.764608
                      12th       84.317673
                      9th        83.939778
Wright High School    10th       83.812757
                      11th       84.156322
                      12th       84.073171
                      9th        83.833333

### Score by School Spending

In [110]:
District_group["Per Student Budget"].min()

array([ 57800.])

In [62]:
#creating bins to place the values
bins=[56500,58000,59500,61000,62500,64000,65500]
#creating group labels
group_labels=["56500 to 58000","58000 to 59500","59500 to 61000","61000 to 62500","62500 to 64000","64000 to 65500"]

In [63]:
#slicing based on per student budget
pd.cut(District_group["Per Student Budget"],bins,labels=group_labels).head()


school
Bailey High School      62500 to 64000
Cabrera High School     58000 to 59500
Figueroa High School    62500 to 64000
Ford High School        64000 to 65500
Griffin High School     61000 to 62500
Name: Per Student Budget, dtype: category
Categories (6, object): [56500 to 58000 < 58000 to 59500 < 59500 to 61000 < 61000 to 62500 < 62500 to 64000 < 64000 to 65500]

In [74]:
District_group["Per Student Budget Group"]=pd.cut(District_group["Per Student Budget"],bins,labels=group_labels)

In [75]:
District_group.head()

,type,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate,Per Student Budget,Per Student Budget Group
school,,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,81.933280,66.680064,74.306672,62800.0,62500 to 64000
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,97.039828,94.133477,95.586652,58200.0,58000 to 59500
Figueroa High School,District,2949,1884411,76.711767,81.158020,80.739234,65.988471,73.363852,63900.0,62500 to 64000
Ford High School,District,2739,1763916,77.102592,80.746258,79.299014,68.309602,73.804308,64400.0,64000 to 65500
Griffin High School,Charter,1468,917500,83.351499,83.816757,97.138965,93.392371,95.265668,62500.0,61000 to 62500


In [78]:

Per_Student_Budget_group=District_group[["math_score","reading_score","%Passing Math","%Passing Reading","Overall Passing Rate","Per Student Budget Group"]]

In [79]:
Per_Student_Budget_group.head()

,math_score,reading_score,%Passing Math,%Passing Reading,Overall Passing Rate,Per Student Budget Group
school,,,,,,
Bailey High School,77.048432,81.033963,66.680064,81.933280,74.306672,62500 to 64000
Cabrera High School,83.061895,83.975780,94.133477,97.039828,95.586652,58000 to 59500
Figueroa High School,76.711767,81.158020,65.988471,80.739234,73.363852,62500 to 64000
Ford High School,77.102592,80.746258,68.309602,79.299014,73.804308,64000 to 65500
Griffin High School,83.351499,83.816757,93.392371,97.138965,95.265668,61000 to 62500


In [80]:
#grouping by perstudent budget group
Scores_by_school_spending =Per_Student_Budget_group.groupby(["Per Student Budget Group"])

In [81]:
Scores_by_school_spending.mean()

,math_score,reading_score,%Passing Math,%Passing Reading,Overall Passing Rate
Per Student Budget Group,,,,,
56500 to 58000,83.274201,83.989488,93.867718,96.539641,95.203679
58000 to 59500,83.515798,83.915256,93.324222,96.634622,94.979422
59500 to 61000,83.599686,83.885211,94.230858,95.900287,95.065572
61000 to 62500,83.351499,83.816757,93.392371,97.138965,95.265668
62500 to 64000,78.505315,81.696400,73.076824,85.050359,79.063592
64000 to 65500,77.023555,80.957446,66.701010,80.675217,73.688113


### Scores by School type


In [121]:
#Selecting the desired columns
School_Type=District_group[["math_score","reading_score","%Passing Math","%Passing Reading","Overall Passing Rate","type"]]


In [122]:

Scores_by_School_Type=School_Type.groupby("type")

In [123]:
Scores_by_School_Type.mean()

,math_score,reading_score,%Passing Math,%Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


### Scores by School Size

In [125]:
School_Size=District_School.groupby("school").agg({"size":"first",
                                                                     "name":"count",
                                                                     "budget":"first",
                                                                     "math_score":"mean",
                                                                     "reading_score":["mean"],
                                                                     "%Passing Reading":["sum"],
                                                                     "%Passing Math":sum })

In [126]:
School_Size.head()

,size,name,budget,math_score,reading_score,%Passing Reading,%Passing Math
,first,count,first,mean,mean,sum,sum
school,,,,,,,
Bailey High School,4976,4976,3124928,77.048432,81.033963,4077.0,3318.0
Cabrera High School,1858,1858,1081356,83.061895,83.975780,1803.0,1749.0
Figueroa High School,2949,2949,1884411,76.711767,81.158020,2381.0,1946.0
Ford High School,2739,2739,1763916,77.102592,80.746258,2172.0,1871.0
Griffin High School,1468,1468,917500,83.351499,83.816757,1426.0,1371.0


In [127]:
#removing the groupby aggregate column
School_Size.columns=School_Size.columns.droplevel(1)

In [128]:
#Calculating the percentage of passing math
School_Size["%Passing Math"]=School_Size["%Passing Math"]/School_Size["name"]*100

In [129]:
#Calculating the percentage of passing reading
School_Size["%Passing Reading"]=School_Size["%Passing Reading"]/School_Size["name"]*100

In [130]:
#Adding the overall passing column
School_Size["Overall Passing Rate"]=(School_Size["%Passing Reading"]+School_Size["%Passing Math"])/2

In [132]:
School_Size.head()

,size,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate
school,,,,,,,,
Bailey High School,4976,4976,3124928,77.048432,81.033963,81.933280,66.680064,74.306672
Cabrera High School,1858,1858,1081356,83.061895,83.975780,97.039828,94.133477,95.586652
Figueroa High School,2949,2949,1884411,76.711767,81.158020,80.739234,65.988471,73.363852
Ford High School,2739,2739,1763916,77.102592,80.746258,79.299014,68.309602,73.804308
Griffin High School,1468,1468,917500,83.351499,83.816757,97.138965,93.392371,95.265668


In [87]:
#identifying the min and max of school size
School_Size["size"].max()

4976

In [162]:
School_Size["size"].min()

array([427], dtype=int64)

In [133]:
#creating bins to place the values
bins=[0,1000,2000,5000]
#creating group labels
group_labels=["Small=<1000","Medium=1000-2000","Large=2000-5000"]

In [135]:
#Selecting desired columns
School_Size_Cut=School_Size[["math_score","reading_score","%Passing Math","%Passing Reading","Overall Passing Rate","size"]]

In [117]:
#slicing based on per student budget
pd.cut(School_Size["size"],bins,labels=group_labels).head(10)

school
Bailey High School        Large=2000-5000
Cabrera High School      Medium=1000-2000
Figueroa High School      Large=2000-5000
Ford High School          Large=2000-5000
Griffin High School      Medium=1000-2000
Hernandez High School     Large=2000-5000
Holden High School            Small=<1000
Huang High School         Large=2000-5000
Johnson High School       Large=2000-5000
Pena High School              Small=<1000
Name: size, dtype: category
Categories (3, object): [Small=<1000 < Medium=1000-2000 < Large=2000-5000]

In [136]:
School_Size["size group"]=pd.cut(School_Size["size"],bins,labels=group_labels)

In [137]:
#grouping by school size
School_Size_group=School_Size.groupby("size group")

In [138]:
School_Size_group.mean()

,size,name,budget,math_score,reading_score,%Passing Reading,%Passing Math,Overall Passing Rate
size group,,,,,,,,
Small=<1000,694.500,694.500,416972.500,83.821598,83.929843,96.099437,93.550225,94.824831
Medium=1000-2000,1704.400,1704.400,1029597.200,83.374684,83.864438,96.790680,93.599695,95.195187
Large=2000-5000,3657.375,3657.375,2333437.125,77.746417,81.344493,82.766634,69.963361,76.364998
